# Comprehensive SyftBox Permissions Tutorial
## Part 1: Fundamentals and Core Concepts

Welcome to the comprehensive guide to SyftBox permissions! This tutorial series will take you from beginner to expert, covering every aspect of how SyftBox's permission system works.

### What You'll Learn in This Series

1. **Part 1 (This notebook): Fundamentals** - Core concepts, permission levels, basic inheritance
2. **Part 2: Patterns and Matching** - Glob patterns, specificity, wildcards, and double-star patterns
3. **Part 3: Inheritance and Hierarchy** - How permissions flow through directory trees
4. **Part 4: Terminal Nodes and Blocking** - Advanced inheritance control
5. **Part 5: File Limits and Restrictions** - Size limits, file type controls
6. **Part 6: Complex Scenarios** - Real-world edge cases and debugging

### Prerequisites
- Basic Python knowledge
- Familiarity with file systems and directories
- Have completed the "5-minute quickstart" tutorial

## Setup

In [ ]:
import syft_perm
from pathlib import Path
import tempfile
import shutil
import yaml

# Create a clean workspace for this tutorial
tutorial_dir = Path(tempfile.mkdtemp(prefix="syftbox_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    """Clean up and create fresh workspace"""
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="syftbox_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path):
    """Display the generated syft.pub.yaml file"""
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        print(f"\n=== {yaml_file} ===")
        print(yaml_file.read_text())
        print("=" * 50)
    else:
        print(f"No syft.pub.yaml in {path}")

## Chapter 1: Understanding Permission Levels

SyftBox has **4 permission levels** arranged in a hierarchy. Higher levels include all permissions of lower levels.

In [ ]:
# Create a test file
test_file = tutorial_dir / "example.txt"
test_file.write_text("Example content")

syft_file = syft_perm.open(test_file)

# The 4 permission levels (lowest to highest)
print("=== Permission Hierarchy ===")
print("1. READ - Can view file contents")
print("2. CREATE - Can read + create new files")
print("3. WRITE - Can read + create + modify existing files")
print("4. ADMIN - Can read + create + write + manage permissions")
print("\nHigher levels include all lower level permissions!")

In [ ]:
# Demonstrate permission hierarchy
syft_file.grant_write_access("alice@example.com", force=True)

print("Alice has WRITE permission, so she also gets:")
print(f"- Read access: {syft_file.has_read_access('alice@example.com')}")
print(f"- Create access: {syft_file.has_create_access('alice@example.com')}")
print(f"- Write access: {syft_file.has_write_access('alice@example.com')}")
print(f"- Admin access: {syft_file.has_admin_access('alice@example.com')}")

syft_file

## Chapter 2: The syft.pub.yaml File Structure

Let's understand what syft-perm generates behind the scenes.

In [ ]:
# Look at the generated YAML
show_yaml(tutorial_dir)

# The structure is:
print("\n=== YAML Structure ===")
print("rules:                    # List of permission rules")
print("- pattern: 'example.txt'  # Which files this rule applies to")
print("  access:                 # Permission grants")
print("    write:                # Permission level")
    print("    - alice@example.com   # Users with this permission")

## Chapter 3: Multiple Users and Permission Levels

In [ ]:
# Grant different permissions to different users
syft_file.grant_read_access("reader@example.com", force=True)
syft_file.grant_create_access("creator@example.com", force=True)
syft_file.grant_admin_access("admin@example.com", force=True)

print("=== Permission Matrix ===")
users = ["reader@example.com", "creator@example.com", "alice@example.com", "admin@example.com"]
permissions = ["read", "create", "write", "admin"]

# Create a table
print(f"{'User':<20} {'Read':<6} {'Create':<8} {'Write':<6} {'Admin':<6}")
print("-" * 55)

for user in users:
    row = f"{user:<20}"
    for perm in permissions:
        has_perm = getattr(syft_file, f"has_{perm}_access")(user)
        row += f" {'✓' if has_perm else '✗':<6}"
    print(row)

syft_file

In [ ]:
# Look at the updated YAML - notice how permissions accumulate
show_yaml(tutorial_dir)

## Chapter 4: Public Access with "*"

The special `"*"` user means "everyone".

In [ ]:
# Create a public file
cleanup_and_reset()
public_file = tutorial_dir / "public.txt"
public_file.write_text("Public information")

syft_public = syft_perm.open(public_file)
syft_public.grant_read_access("*", force=True)  # Everyone can read

# Test with random users
test_users = ["anyone@example.com", "random@user.com", "test@domain.org"]
print("Public access test:")
for user in test_users:
    can_read = syft_public.has_read_access(user)
    print(f"{user}: {can_read}")

show_yaml(tutorial_dir)

## Chapter 5: Folder Permissions and Basic Inheritance

Permissions can be applied to folders, and files inherit permissions from their parent directories.

In [ ]:
# Create a folder structure
cleanup_and_reset()
project_dir = tutorial_dir / "my_project"
project_dir.mkdir()

# Create files in the folder
files = [
    project_dir / "README.md",
    project_dir / "main.py",
    project_dir / "data.csv"
]

for file_path in files:
    file_path.write_text(f"Content of {file_path.name}")

print(f"Created folder: {project_dir}")
print(f"Created files: {[f.name for f in files]}")

In [ ]:
# Apply permissions to the FOLDER
syft_folder = syft_perm.open(project_dir)
syft_folder.grant_read_access("team@example.com", force=True)
syft_folder.grant_write_access("lead@example.com", force=True)

print("Folder permissions:")
syft_folder

In [ ]:
# Check that files INHERIT permissions from the folder
print("\n=== Inheritance Test ===")
for file_path in files:
    syft_file = syft_perm.open(file_path)
    
    team_read = syft_file.has_read_access("team@example.com")
    lead_write = syft_file.has_write_access("lead@example.com")
    
    print(f"{file_path.name}:")
    print(f"  Team can read: {team_read}")
    print(f"  Lead can write: {lead_write}")
    print()

In [ ]:
# Look at the YAML - only the folder has rules, files inherit
show_yaml(tutorial_dir)
show_yaml(project_dir)  # Folder has the rules

## Chapter 6: Permission Debugging and Reasoning

Understanding WHY a permission was granted or denied is crucial for debugging.

In [ ]:
# Let's debug the inheritance we just saw
readme_file = syft_perm.open(project_dir / "README.md")

print("=== Detailed Permission Analysis ===")
explanation = readme_file.explain_permissions("team@example.com")
print(explanation)

In [ ]:
# Check specific permission with reasons
has_write, reasons = readme_file._check_permission_with_reasons("team@example.com", "write")
print(f"Team can write README.md: {has_write}")
print("Reasons:")
for reason in reasons:
    print(f"  - {reason}")

## Chapter 7: The Nearest-Node Algorithm

This is the **core concept** of SyftBox permissions! When checking permissions for a file, SyftBox finds the **nearest parent directory** that has matching rules.

In [ ]:
# Create a multi-level directory structure
cleanup_and_reset()

# Structure:
# tutorial_dir/
#   ├── parent/
#   │   ├── child/
#   │   │   └── file.txt
#   │   └── sibling.txt
#   └── other.txt

parent_dir = tutorial_dir / "parent"
child_dir = parent_dir / "child"
child_dir.mkdir(parents=True)

# Create files at different levels
files = {
    "root_file": tutorial_dir / "other.txt",
    "parent_file": parent_dir / "sibling.txt", 
    "child_file": child_dir / "file.txt"
}

for name, path in files.items():
    path.write_text(f"Content of {name}")

print("Created directory structure:")
print(f"{tutorial_dir}/")
print(f"├── parent/")
print(f"│   ├── child/")
print(f"│   │   └── file.txt")
print(f"│   └── sibling.txt")
print(f"└── other.txt")

In [ ]:
# Set permissions only on the PARENT directory
syft_parent = syft_perm.open(parent_dir)
syft_parent.grant_read_access("alice@example.com", force=True)

print("Set permissions on parent/ directory only")
show_yaml(parent_dir)

In [ ]:
# Test the nearest-node algorithm
print("=== Nearest-Node Algorithm Test ===")

for name, path in files.items():
    syft_file = syft_perm.open(path)
    can_read = syft_file.has_read_access("alice@example.com")
    
    print(f"{name} ({path.relative_to(tutorial_dir)}):")
    print(f"  Alice can read: {can_read}")
    
    # Show the reasoning
    has_read, reasons = syft_file._check_permission_with_reasons("alice@example.com", "read")
    print(f"  Reason: {reasons[0] if reasons else 'No reason'}")
    print()

**Key Insight**: 
- `file.txt` and `sibling.txt` inherit from `parent/` (nearest node with rules)
- `other.txt` has no permissions because there's no matching rule in any parent

## Chapter 8: Ownership and Automatic Permissions

File owners automatically get all permissions, regardless of explicit rules.

In [ ]:
# Create a datasite structure (simulates SyftBox ownership)
cleanup_and_reset()

# SyftBox determines ownership by path: datasites/{email}/...
datasite_dir = tutorial_dir / "datasites" / "alice@example.com"
datasite_dir.mkdir(parents=True)

# Alice's file in her datasite
alice_file = datasite_dir / "my_file.txt"
alice_file.write_text("Alice's private data")

syft_alice_file = syft_perm.open(alice_file)

# Test ownership detection
print("=== Ownership Test ===")
print(f"Alice owns her file: {syft_alice_file.has_admin_access('alice@example.com')}")
print(f"Bob can't access: {syft_alice_file.has_read_access('bob@example.com')}")

# Show reasoning
has_admin, reasons = syft_alice_file._check_permission_with_reasons("alice@example.com", "admin")
print(f"\nWhy Alice has admin access: {reasons}")

## Chapter 9: Revoking Permissions

In [ ]:
# Create a file with multiple users
cleanup_and_reset()
shared_file = tutorial_dir / "shared.txt"
shared_file.write_text("Shared content")

syft_shared = syft_perm.open(shared_file)

# Grant permissions to multiple users
users = ["alice@example.com", "bob@example.com", "charlie@example.com"]
for user in users:
    syft_shared.grant_read_access(user, force=True)

print("Before revocation:")
syft_shared

In [ ]:
# Revoke Bob's access
syft_shared.revoke_read_access("bob@example.com")

print("After revoking Bob's access:")
for user in users:
    can_read = syft_shared.has_read_access(user)
    print(f"{user}: {can_read}")

syft_shared

In [ ]:
# Revoke public access
syft_shared.grant_read_access("*", force=True)  # Grant public access first
print("After granting public access:")
print(f"Random user can read: {syft_shared.has_read_access('random@example.com')}")

syft_shared.revoke_read_access("*")  # Revoke public access
print("\nAfter revoking public access:")
print(f"Random user can read: {syft_shared.has_read_access('random@example.com')}")
print(f"Alice still can read: {syft_shared.has_read_access('alice@example.com')}")

## Summary: Part 1 Key Concepts

Congratulations! You've learned the fundamentals:

### ✅ **Core Concepts Mastered**
1. **Permission Hierarchy**: Read < Create < Write < Admin
2. **Higher includes lower**: Write permission includes Read and Create
3. **Public access**: Use `"*"` for everyone
4. **Inheritance**: Files inherit permissions from parent directories
5. **Nearest-node algorithm**: Find the closest parent with matching rules
6. **Ownership**: File owners get all permissions automatically
7. **YAML structure**: `rules` → `pattern` → `access` → permission levels
8. **Debugging**: Use `explain_permissions()` and `_check_permission_with_reasons()`

### 🚀 **Next Up: Part 2 - Patterns and Matching**
In the next tutorial, you'll learn:
- Glob patterns (`*.py`, `docs/**`, `**/*.txt`)
- Pattern specificity and conflict resolution
- Wildcard matching rules
- Advanced pattern debugging

The pattern system is where SyftBox permissions become truly powerful!

In [ ]:
# Cleanup
shutil.rmtree(tutorial_dir)
print("Tutorial 1 complete! 🎉")